In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.2 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 3.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 5.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 10.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 12.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.6 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-bz8yjqg2/overlay/local/bin
    changing mode of /tmp/pip-build-env-bz8yjqg2/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-bz8yjqg2/overlay/loc

In [4]:
import re
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [2]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [5]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [6]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=1024, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [7]:
from google.colab import files
pd= files.upload()

Saving processed_reviews_128 (6).csv to processed_reviews_128 (6).csv


In [8]:
import pandas as pd
import random
df = pd.read_csv('processed_reviews_128 (6).csv')


In [9]:
reviews_df = df[['sentiment', 'cleaned_review']]
reviews_df

,sentiment,cleaned_review
0,positive,mani funni scene peopl dont normal pay attent ...
1,negative,shame shame fine actor joseph fienn would allo...
2,negative,movi pretti predict nuff saidfrom delay kiss s...
3,positive,black mask jet li play bioengin superkil turn ...
4,negative,watch entir movi recogn particip william hurt ...
...,...,...
1995,negative,horribl mish mash predict stori line toebendin...
1996,negative,thoughtless ignor illconceiv careerkil talent ...
1997,positive,saw film ae channel past weekend mysteri okay ...
1998,negative,home weekend bore lack move absolut noth bette...


In [10]:
#cetroid
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import numpy as np


In [11]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews_df['cleaned_review'])

In [12]:
positive_indices = reviews_df[reviews_df['sentiment'] == 'positive'].index
negative_indices = reviews_df[reviews_df['sentiment'] == 'negative'].index

In [13]:
def calculate_centroids(df, X, sentiment_column='sentiment'):
    positive_indices = df[df[sentiment_column] == 'positive'].index
    negative_indices = df[df[sentiment_column] == 'negative'].index

    positive_vectors = X[positive_indices]
    negative_vectors = X[negative_indices]

    positive_centroid = np.mean(positive_vectors, axis=0)
    negative_centroid = np.mean(negative_vectors, axis=0)

    return positive_centroid, negative_centroid

In [14]:
positive_centroid, negative_centroid = calculate_centroids(reviews_df, X)

In [16]:
def find_closest_to_centroid(indices, centroid, X):
    centroid = np.asarray(centroid)

    distances = pairwise_distances(X[indices], centroid, metric='euclidean')

    closest_index = np.argmin(distances)

    return indices[closest_index]

In [17]:
closest_positive_review_index = find_closest_to_centroid(positive_indices, positive_centroid, X)
closest_negative_review_index = find_closest_to_centroid(negative_indices, negative_centroid, X)
closest_positive_review = reviews_df.loc[closest_positive_review_index, 'cleaned_review']
closest_negative_review = reviews_df.loc[closest_negative_review_index, 'cleaned_review']

print("Текст ближайшего положительного отзыва:", closest_positive_review)
print("Текст ближайшего отрицательного отзыва:", closest_negative_review)

Текст ближайшего положительного отзыва: watch film tv awesom br br plan watch whilst work howev end watch whole film work good br br actor well know stori line make fact actor make less well known make believ event could occur feel bias toward one charact judgement type charact never seen actor film made even enjoyablebr br would recommend watch
Текст ближайшего отрицательного отзыва: terribl movi one wast time go see someth els movi without doubt one worst movi ever seen life want see good movi dont see made men


In [18]:
df_incorrect = df.loc[df['sentiment'] == 'negative']['cleaned_review'].reset_index(drop=True)
df_correct = df.loc[df['sentiment'] == 'positive']['cleaned_review'].reset_index(drop=True)
df_correct[random.choice(range(len(df_correct)))]

'excel cast make movi work charact develop exceedingli well clear actor enjoy film moviebr br quit comedi expect much lightheart look attempt reclaim youth glori bawdi humor music fan quit subtl refer intellig funnybr br hate draw direct comparison movi much movi remind alan parker film cant help imagin commit actual make big tri recaptur said glori year later'

In [19]:
df_incorrect[random.choice(range(len(df_incorrect)))]

'first jenna jameson best actress movi she aw movi everi horror move clich imagin badli play oversex teen coupl comicalnothorni jock snotti cool chick creepi local color guy parentaltyp coupl virgin chick amazingli never die film dialogu pain aw deliv depth wade pool almost like your wish theyd die sooner saw rough cut film ago somehow got wors sure funniest thing ghoul tri eat jameson implant hardli rate even rental dog avoid cost'

In [20]:
df_correct_ = 'peopl call comedi watch laughedbr br guess problem first saw br br wasnt old enough understand wasnt meant bebr br taken serious quit scene meantbr br funni care much charact laugh atbr br thembr br suggest watch film next time your fall lovebr br tri take serious think youll find despit sillybr br flaw one move love stori youv ever seen'
df_incorrect_ = 'read review decid rent dvd version br br like classic music want learn bach br br disappoint guess know enough bach music comment enough understand import music play br br mayb would appropri guidanc expert bach music explain film br br realli tri saw whole film hop would abl enjoy least br br see risk'

In [ ]:
#Step 6: Generating the Response

In [21]:
def generate_prompt(df, index):
    return f'''Examples of movie review:
    1. {closest_positive_review}
     ANSWER: positive
    2. {closest_negative_review}
     ANSWER: negative

    Please read and analyze both reviews. Read and correlate the 3rd review to either the 1st or 2nd review based on emotional tone.:
    3th review: "{df['cleaned_review'][index][:128]}"
    '''

END_INDEX = 30

prompts = []
model_r = []
model_responses = []
review_index = 0

while review_index < END_INDEX:
    prompt = generate_prompt(df, review_index)

    prompt_template = f'''SYSTEM:You are a movie review evaluator. Your task is to correlate the 3rd review to either the 1st or 2nd review based on emotional tone

    USER: {prompt}

    ASSISTANT:
    '''

    response = lcpp_llm(prompt=prompt_template, max_tokens=512, temperature=0.5, top_p=0.95,
                      repeat_penalty=1.2, top_k=150,
                      echo=True)

    print(response)
    print(response["choices"][0]["text"])

    response_text = response["choices"][0]["text"]

    matches_ = re.search(r'ASSISTANT:(.*?)(?=Model response|$)', response_text, re.DOTALL)

    if matches_:
        assistant_response = matches_.group(1).strip()
        answer_match = re.search(r'\b(?:1(?:st|\.|rst)?|first|2(?:nd|\.|cond)?|second)\b', assistant_response, re.IGNORECASE)

        # Ensure that only one value is appended for each iteration to `model_responses`
        matched_review_index = answer_match.group(0) if answer_match else None
        model_responses.append(matched_review_index)

        if answer_match:  # Append only when there is a match
            matched_review_index = 1 if answer_match.group().lower() in ['1st', '1.', '1', 'first'] else 2
            prompts.append(prompt_template)
            model_r.append(assistant_response)
        else:  # Append None or placeholders when there's no match
            prompts.append(prompt_template)
            model_r.append(None)  # Or some placeholder that indicates no valid response was found
    else:
        print("Model response does not contain a valid assistant response. Retrying...")
        # Append None or placeholders in this case too
        prompts.append(prompt_template)
        model_r.append(None)
        model_responses.append(None)

    review_index += 1

df_results = pd.DataFrame({
    'Prompt': prompts,
    'Model': model_r,
    'Model Response': model_responses,
})




{'id': 'cmpl-accd1017-f810-4f91-8439-42f0922b0174', 'object': 'text_completion', 'created': 1704927821, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a movie review evaluator. Your task is to correlate the 3rd review to either the 1st or 2nd review based on emotional tone\n\n    USER: Examples of movie review:\n    1. watch film tv awesom br br plan watch whilst work howev end watch whole film work good br br actor well know stori line make fact actor make less well known make believ event could occur feel bias toward one charact judgement type charact never seen actor film made even enjoyablebr br would recommend watch\n     ANSWER: positive\n    2. terribl movi one wast time go see someth els movi without doubt one worst movi ever seen life want see good movi dont see made men\n     ANSWER: negative\n\n    Please read and analyze 

Llama.generate: prefix-match hit


{'id': 'cmpl-90d551f0-23f3-4ba1-816d-98b547ad83a8', 'object': 'text_completion', 'created': 1704927894, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a movie review evaluator. Your task is to correlate the 3rd review to either the 1st or 2nd review based on emotional tone\n\n    USER: Examples of movie review:\n    1. watch film tv awesom br br plan watch whilst work howev end watch whole film work good br br actor well know stori line make fact actor make less well known make believ event could occur feel bias toward one charact judgement type charact never seen actor film made even enjoyablebr br would recommend watch\n     ANSWER: positive\n    2. terribl movi one wast time go see someth els movi without doubt one worst movi ever seen life want see good movi dont see made men\n     ANSWER: negative\n\n    Please read and analyze 

Llama.generate: prefix-match hit


{'id': 'cmpl-6d9cd0da-c8f8-4b9e-8cf0-2cc100f2ef7a', 'object': 'text_completion', 'created': 1704927945, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a movie review evaluator. Your task is to correlate the 3rd review to either the 1st or 2nd review based on emotional tone\n\n    USER: Examples of movie review:\n    1. watch film tv awesom br br plan watch whilst work howev end watch whole film work good br br actor well know stori line make fact actor make less well known make believ event could occur feel bias toward one charact judgement type charact never seen actor film made even enjoyablebr br would recommend watch\n     ANSWER: positive\n    2. terribl movi one wast time go see someth els movi without doubt one worst movi ever seen life want see good movi dont see made men\n     ANSWER: negative\n\n    Please read and analyze 

...

{'id': 'cmpl-e24eaa48-35a7-4779-a7e7-36e1e813d7db', 'object': 'text_completion', 'created': 1704929459, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a movie review evaluator. Your task is to correlate the 3rd review to either the 1st or 2nd review based on emotional tone\n\n    USER: Examples of movie review:\n    1. watch film tv awesom br br plan watch whilst work howev end watch whole film work good br br actor well know stori line make fact actor make less well known make believ event could occur feel bias toward one charact judgement type charact never seen actor film made even enjoyablebr br would recommend watch\n     ANSWER: positive\n    2. terribl movi one wast time go see someth els movi without doubt one worst movi ever seen life want see good movi dont see made men\n     ANSWER: negative\n\n    Please read and analyze 

Llama.generate: prefix-match hit


{'id': 'cmpl-1f768ad1-2f57-4a64-b108-9dbc985478a1', 'object': 'text_completion', 'created': 1704929556, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a movie review evaluator. Your task is to correlate the 3rd review to either the 1st or 2nd review based on emotional tone\n\n    USER: Examples of movie review:\n    1. watch film tv awesom br br plan watch whilst work howev end watch whole film work good br br actor well know stori line make fact actor make less well known make believ event could occur feel bias toward one charact judgement type charact never seen actor film made even enjoyablebr br would recommend watch\n     ANSWER: positive\n    2. terribl movi one wast time go see someth els movi without doubt one worst movi ever seen life want see good movi dont see made men\n     ANSWER: negative\n\n    Please read and analyze 

Llama.generate: prefix-match hit


{'id': 'cmpl-e0859d9b-1ddf-4011-8ee0-50c3b8e156e8', 'object': 'text_completion', 'created': 1704929609, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a movie review evaluator. Your task is to correlate the 3rd review to either the 1st or 2nd review based on emotional tone\n\n    USER: Examples of movie review:\n    1. watch film tv awesom br br plan watch whilst work howev end watch whole film work good br br actor well know stori line make fact actor make less well known make believ event could occur feel bias toward one charact judgement type charact never seen actor film made even enjoyablebr br would recommend watch\n     ANSWER: positive\n    2. terribl movi one wast time go see someth els movi without doubt one worst movi ever seen life want see good movi dont see made men\n     ANSWER: negative\n\n    Please read and analyze 

In [22]:
df_results

,Prompt,Model,Model Response
0,SYSTEM:You are a movie review evaluator. Your ...,"Okay, I've read and analyzed both reviews. Bas...",1st
1,SYSTEM:You are a movie review evaluator. Your ...,"Based on the 3rd review, I would correlate it ...",1st
2,SYSTEM:You are a movie review evaluator. Your ...,"Based on the emotional tone of the 3rd review,...",1st
3,SYSTEM:You are a movie review evaluator. Your ...,I have read and analyzed both reviews. Based o...,2nd
4,SYSTEM:You are a movie review evaluator. Your ...,Hi there! I'm here to help you with your quest...,first
5,SYSTEM:You are a movie review evaluator. Your ...,"Okay, I've read and analyzed both reviews. Bas...",1st
6,SYSTEM:You are a movie review evaluator. Your ...,"Based on the reviews you provided, I would say...",1st
7,SYSTEM:You are a movie review evaluator. Your ...,Hey there! I've got the 3rd review here and I'...,1st
8,SYSTEM:You are a movie review evaluator. Your ...,"Based on the reviews you provided, I would say...",second
9,SYSTEM:You are a movie review evaluator. Your ...,"Based on the emotional tone of the 3rd review,...",2


In [23]:
df_results.to_csv('K_MEANS_5.csv', index=False)
files.download('K_MEANS_5.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
!pip install scikit-learn

true_labels = [1,0,0,1,0,0,1,0,0,0,1,0,0,1,1,1,0,0,1,0,1,1,1,0,1,1,1,1,1]
predicted_labels = [1,1,1,0,1,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,1,1,1,0,1,1,1,1,0]

correct_predictions = sum(1 for true, predicted in zip(true_labels, predicted_labels) if true == predicted)

accuracy = correct_predictions / len(true_labels)

print(f"Точность модели: {accuracy * 100:.2f}%")

Точность модели: 58.62%
